In [ ]:
# import required libraries from titanic example
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer, LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import warnings
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import torch.optim as optim
from PIL import Image
import os
warnings.filterwarnings("ignore")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

100%|██████████| 658M/658M [00:04<00:00, 143MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1


Creating a custom data loader class

In [ ]:
class PlantDataset(Dataset):
    """Custom Dataset for loading plant disease images"""
    def __init__(self, imagepaths, labels, transform=None):
        self.imagepaths = imagepaths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.imagepaths[idx]
        label = self.labels[idx]

        # Load the image and apply transformations
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.long)

In [ ]:
path = path+'/PlantVillage'

Load and Prepare Data

In [ ]:
image_paths, labels = [], []

for disease in os.listdir(path):
    disease_path = os.path.join(path, disease)

    for image in os.listdir(disease_path):
      if image.startswith("."):
        continue
      image_path = os.path.join(disease_path, image)
      if image_path.lower().endswith((".jpg", ".jpeg", ".png")):
        image_paths.append(image_path)
        labels.append(disease)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

class_names = list(label_encoder.classes_)

In [ ]:
print(image_paths[:5])
print(labels[5:])

['/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/1666ae7f-31f9-4faf-8b60-40a1ded014ea___YLCV_GCREC 2236.JPG', '/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/4254afe0-7830-4ffa-bd0d-5d298280275c___UF.GRC_YLCV_Lab 01523.JPG', '/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/7e3bc5f6-9890-4e2e-90a3-cd394eedb6c5___YLCV_NREC 2412.JPG', '/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/13fb8aaa-4db1-405e-86b9-dc482b10b7d8___UF.GRC_YLCV_Lab 02339.JPG', '/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/1559bd4d-251e-4568-9e25-08c1e8fa3674___YLCV_NREC 2384.JPG']
[12 12 12 12 12]


In [ ]:
#train test split
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


print("Train size:", len(train_paths))
print("Test size:", len(test_paths))


Train size: 16510
Test size: 4128


In [ ]:
#apply transformations to avoid model to avoid overfit

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = PlantDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PlantDataset(test_paths, test_labels, transform=valid_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class ImageClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 256, 5)
        self.conv3 = nn.Conv2d(256, 128, 5)
        self.conv4 = nn.Conv2d(128, 16, 5)
        self.fc1 = nn.Linear(1600, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageClassifier(num_classes=len(class_names)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(3):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if i % 100 == 0:
            print(f"Epoch {epoch+1}/{10}, Batch {i}/{len(train_loader)}, Loss: {loss.item()}")
            train_loss = 0.0



Epoch 1/10, Batch 0/516, Loss: 2.684429883956909
Epoch 1/10, Batch 100/516, Loss: 1.9450310468673706
Epoch 1/10, Batch 200/516, Loss: 1.882965087890625
Epoch 1/10, Batch 300/516, Loss: 1.393473744392395
Epoch 1/10, Batch 400/516, Loss: 1.1970959901809692
Epoch 1/10, Batch 500/516, Loss: 1.1970373392105103
Epoch 2/10, Batch 0/516, Loss: 0.9963145852088928
Epoch 2/10, Batch 100/516, Loss: 0.9472709894180298
Epoch 2/10, Batch 200/516, Loss: 1.4212031364440918
Epoch 2/10, Batch 300/516, Loss: 0.862163782119751
Epoch 2/10, Batch 400/516, Loss: 0.7142228484153748
Epoch 2/10, Batch 500/516, Loss: 0.5858836770057678
Epoch 3/10, Batch 0/516, Loss: 0.7702586054801941
Epoch 3/10, Batch 100/516, Loss: 0.8765339255332947
Epoch 3/10, Batch 200/516, Loss: 0.7148088216781616
Epoch 3/10, Batch 300/516, Loss: 0.9208993315696716
Epoch 3/10, Batch 400/516, Loss: 0.926418662071228
Epoch 3/10, Batch 500/516, Loss: 0.6222499012947083


In [ ]:
test_loss = 0.0
correct_predictions = 0
total_samples = 0
all_predictions = []
all_labels = []


with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
accuracy = correct_predictions / total_samples * 100

# Print results
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


Test Loss: 0.7573, Test Accuracy: 74.03%


Task1 Part 2: VGG

In [ ]:
model = torchvision.models.vgg16(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 166MB/s]


In [ ]:
model.train()
for epoch in range(3):
  train_loss = 0.0
  for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if i % 100 == 0:
            print(f"Epoch {epoch+1}/{10}, Batch {i}/{len(train_loader)}, Loss: {loss.item()}")
            train_loss = 0.0



Epoch 1/10, Batch 0/516, Loss: 2.766028642654419
Epoch 1/10, Batch 100/516, Loss: 2.7980244159698486
Epoch 1/10, Batch 200/516, Loss: 2.8107030391693115
Epoch 1/10, Batch 300/516, Loss: 2.9490106105804443
Epoch 1/10, Batch 400/516, Loss: 2.705845832824707
Epoch 1/10, Batch 500/516, Loss: 2.7630910873413086
Epoch 2/10, Batch 0/516, Loss: 2.802272081375122
Epoch 2/10, Batch 100/516, Loss: 2.807744264602661
Epoch 2/10, Batch 200/516, Loss: 2.7819747924804688
Epoch 2/10, Batch 300/516, Loss: 2.843998432159424
Epoch 2/10, Batch 400/516, Loss: 2.887906551361084
Epoch 2/10, Batch 500/516, Loss: 2.7316181659698486
Epoch 3/10, Batch 0/516, Loss: 2.713881254196167
Epoch 3/10, Batch 100/516, Loss: 2.809537172317505
Epoch 3/10, Batch 200/516, Loss: 2.72906231880188
Epoch 3/10, Batch 300/516, Loss: 2.838984251022339
Epoch 3/10, Batch 400/516, Loss: 2.769102096557617
Epoch 3/10, Batch 500/516, Loss: 2.7028422355651855


In [ ]:
test_loss = 0.0
correct_predictions = 0
total_samples = 0
all_predictions = []
all_labels = []


with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
accuracy = correct_predictions / total_samples * 100

# Print results
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")


Test Loss: 2.7880, Test Accuracy: 6.64%


VGG performs far less accurately due to overfit

Part 3: adding diversity to the dataset and repeating part 1

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = PlantDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PlantDataset(test_paths, test_labels, transform=valid_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model = ImageClassifier(num_classes=len(class_names)).to(device)

In [23]:
for epoch in range(3):
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if i % 100 == 0:
            print(f"Epoch {epoch+1}/{10}, Batch {i}/{len(train_loader)}, Loss: {loss.item()}")
            train_loss = 0.0

Epoch 1/10, Batch 0/516, Loss: 2.702105760574341
Epoch 1/10, Batch 100/516, Loss: 2.722494125366211
Epoch 1/10, Batch 200/516, Loss: 2.699586868286133
Epoch 1/10, Batch 300/516, Loss: 2.7081170082092285
Epoch 1/10, Batch 400/516, Loss: 2.7087597846984863
Epoch 1/10, Batch 500/516, Loss: 2.6999173164367676
Epoch 2/10, Batch 0/516, Loss: 2.714127779006958
Epoch 2/10, Batch 100/516, Loss: 2.7021172046661377
Epoch 2/10, Batch 200/516, Loss: 2.6997079849243164
Epoch 2/10, Batch 300/516, Loss: 2.7157630920410156
Epoch 2/10, Batch 400/516, Loss: 2.710273265838623
Epoch 2/10, Batch 500/516, Loss: 2.700863838195801
Epoch 3/10, Batch 0/516, Loss: 2.7259464263916016
Epoch 3/10, Batch 100/516, Loss: 2.7177107334136963
Epoch 3/10, Batch 200/516, Loss: 2.723723888397217
Epoch 3/10, Batch 300/516, Loss: 2.713331460952759
Epoch 3/10, Batch 400/516, Loss: 2.719778537750244
Epoch 3/10, Batch 500/516, Loss: 2.722012758255005


In [24]:
test_loss = 0.0
correct_predictions = 0
total_samples = 0
all_predictions = []
all_labels = []


with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
accuracy = correct_predictions / total_samples * 100

# Print results
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

Test Loss: 2.7070, Test Accuracy: 11.02%


Adding diversity makes model performance worse, as the increased diversity in training data is not applicable to the test data